<a href="https://colab.research.google.com/github/sungshin-ai/ai_teamplay/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
  -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-12-04 14:20:25--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.128, 142.250.159.128, 142.251.161.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   136MB/s    in 0.6s    

2022-12-04 14:20:26 (136 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [6]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input #InceptionV3, Xception, InceptionResNetV2
#from keras.applications.imagenet_utils import preprocess_input #others
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization

from tensorflow import keras
from PIL import Image
import tensorflow as tf

from keras.callbacks import ModelCheckpoint

In [3]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                              include_top=False,
                              weights=None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [4]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output.shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [14]:
#from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

#model.compile(optimizer=RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [7]:
select_df=pd.read_csv('/content/drive/MyDrive/DF_select.csv')
select_df

,name,fold,class,num,path,label
0,00_01.0045.jpg,00_01,1,45,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
1,00_01.0046.jpg,00_01,1,46,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
2,00_01.0047.jpg,00_01,1,47,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
3,00_01.0048.jpg,00_01,1,48,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
4,00_01.0049.jpg,00_01,1,49,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
...,...,...,...,...,...,...
3595,19_77.0072.jpg,19_77,77,72,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3596,19_77.0073.jpg,19_77,77,73,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3597,19_77.0074.jpg,19_77,77,74,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3598,19_77.0075.jpg,19_77,77,75,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care


In [8]:
select_df['path']='/content/drive/MyDrive/KSL_rgb/'+select_df['fold']+'/'+select_df['name']

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = train_datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

Found 2700 validated image filenames belonging to 10 classes.
Found 900 validated image filenames belonging to 10 classes.


In [20]:
checkpoint_filepath = '/content/drive/MyDrive/keras/checkpoint/best_tfl.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [ ]:
history = model.fit(
  train_generator,
  validation_data=valid_generator,
  steps_per_epoch=100,
  epochs=10,
  validation_steps=50,
  verbose=1,
  callbacks=[model_checkpoint_callback]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()